# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding-10724000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo-6131000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Running through an example

In [2]:
# EW vulnerable.
vuln_ns, vuln_ew = False, True
# you sit North and hold:
hand = 'QT5.T8.KQT87.864'
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 0, False)

# the auction goes:
auction = ["PAD_START","PAD_START","PAD_START","PASS"]
bid = bot_bid.bid(auction)

auction = ["PAD_START","PAD_START","PAD_START","PASS", "PASS", "1N", "2C", "PASS"]
# what's your bid?
bid = bot_bid.bid(auction)


bid PASS value 0.9993 is recommended by NN
0 ['PAD_START', 'PAD_START', 'PAD_START', 'PASS']
sample_cards_auction, nsteps= 1
NS:  1.0 EW:  99.0 Auction:  ['PAD_START', 'PAD_START', 'PAD_START', 'PASS']
nesw_i 0
Player:  N Hand:  QTx.T8.KQT8x.8xx
HCP:  [[12.71 13.18  7.81]]
Shape:  [[3.05 3.8  2.71 3.33 3.09 3.74 2.78 3.23 3.09 3.69 2.8  3.35]]
[[[ 1.   99.    0.    1.   -0.75 -0.14 -0.71  1.   -0.14  0.    0.    1.    0.    1.    0.    0.    1.    0.    0.    0.    0.    1.    0.    1.    0.    0.    1.    1.    0.    1.    0.    1.
    1.    0.    0.    0.    0.    0.    0.    1.    2.    0.    0.    1.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    1.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   

In [3]:
bid.samples

['QTx.T8.KQT8x.8xx 9xx.AJxx.AJ9.AKx A8x.KQx.xx.QJxxx KJxx.9xxx.xxx.T9 0.81083 \n P-P-1N-2C-P-2S-P-P-P (5.4)  \n P-P-1N-2C-P-P-P (7.4)  \n P-P-1N-2C-P-2D-P-P-P (6.6)  \n P-P-1N-2C-P-3C-P-P-P (7.4)  \n P-P-1N-2C-P-2H-P-P-P (4.7)  \n P-P-1N-2C-P-P-P (7.4) ',
 'QTx.T8.KQT8x.8xx AKxx.KQx.J9x.KJx J8x.Axx.Ax.ATxxx 9xx.J9xxx.xxx.Q9 0.79051 \n P-P-1N-2C-P-2S-P-P-P (6.3)  \n P-P-1N-2C-P-P-P (8.1)  \n P-P-1N-2C-P-2D-P-P-P (7.9)  \n P-P-1N-2C-P-3C-P-P-P (8.1)  \n P-P-1N-2C-P-2H-P-P-P (5.3)  \n P-P-1N-2C-P-P-P (8.1) ',
 'QTx.T8.KQT8x.8xx AK8.xxx.Ax.KJxxx Jxxxx.AKJxx.x.A9 9x.Q9x.J9xxx.QTx 0.75569 \n P-P-1N-2C-P-2S-3C-P-P-P (8.0)  \n P-P-1N-2C-P-P-3C-P-3D-P-P-P (6.2)  \n P-P-1N-2C-P-2D-3C-P-P-P (8.0)  \n P-P-1N-2C-P-3C-X-P-P-P (4.7)  \n P-P-1N-2C-P-2H-3C-P-P-P (8.0)  \n P-P-1N-2C-P-P-3C-P-3D-P-P-P (6.2) ',
 'QTx.T8.KQT8x.8xx AK.J9x.AJxx.Kxxx J98xx.KQxxx..AJ9 xxx.Axx.9xxx.QTx 0.74878 \n P-P-1N-2C-P-2S-P-P-P (7.8)  \n P-P-1N-2C-P-P-P (5.5)  \n P-P-1N-2C-P-2D-P-2S-P-P-P (8.0)  \n P-P-1N-2C-P-3C-P-P-P (5

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [4]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp = bot_bid.sample_hands_auction(["PAD_START","PAD_START","PAD_START","PASS", "PASS", "1N", "2C", "PASS"])

for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

0 ['PAD_START', 'PAD_START', 'PAD_START', 'PASS', 'PASS', '1N', '2C', 'PASS']
sample_cards_auction, nsteps= 2
NS:  1.0 EW:  99.0 Auction:  ['PAD_START', 'PAD_START', 'PAD_START', 'PASS', 'PASS', '1N', '2C', 'PASS']
nesw_i 0
Player:  N Hand:  QTx.T8.KQT8x.8xx
HCP:  [[15.47 13.27  4.33]]
Shape:  [[3.14 3.52 3.13 3.24 3.41 4.11 1.88 3.6  3.11 3.36 3.11 3.38]]
[[[ 1.   99.    0.    1.   -0.75 -0.14 -0.71  1.   -0.14  0.    0.    1.    0.    1.    0.    0.    1.    0.    0.    0.    0.    1.    0.    1.    0.    0.    1.    1.    0.    1.    0.    1.
    1.    0.    0.    0.    0.    0.    0.    1.    2.    0.    0.    1.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    1.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  